## 准备数据

In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [16]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [17]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        # self.W1 = tf.compat.v1.get_variable(name="W1", shape=(784, 100), initializer=tf.random_normal_initializer(), trainable=True)
        # self.W2 = tf.compat.v1.get_variable(name="W2", shape=(100, 10), initializer=tf.random_normal_initializer(), trainable=True)
        # self.b1 = tf.compat.v1.get_variable(name="b1", shape=(100, ), trainable=True)
        # self.b2 = tf.compat.v1.get_variable(name="b2", shape=(10, ), trainable=True)
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 100],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,initial_value=tf.random.uniform(shape=[100, 10],minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        # x_prime = tf.reshape(x, [-1, 784])
        # h1 = tf.nn.relu(tf.matmul(x_prime, self.W1)+self.b1)
        # logits = tf.matmul(h1, self.W2)+self.b2
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [18]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [19]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3071432 ; accuracy 0.15588333
epoch 1 : loss 2.3020298 ; accuracy 0.16215
epoch 2 : loss 2.296952 ; accuracy 0.16825
epoch 3 : loss 2.291908 ; accuracy 0.17456667
epoch 4 : loss 2.2868967 ; accuracy 0.18111667
epoch 5 : loss 2.2819169 ; accuracy 0.18708333
epoch 6 : loss 2.2769678 ; accuracy 0.1929
epoch 7 : loss 2.2720482 ; accuracy 0.19923334
epoch 8 : loss 2.2671568 ; accuracy 0.2049
epoch 9 : loss 2.2622936 ; accuracy 0.21083333
epoch 10 : loss 2.2574568 ; accuracy 0.21703333
epoch 11 : loss 2.2526464 ; accuracy 0.22263333
epoch 12 : loss 2.2478604 ; accuracy 0.2287
epoch 13 : loss 2.243099 ; accuracy 0.23476666
epoch 14 : loss 2.238361 ; accuracy 0.24068333
epoch 15 : loss 2.233646 ; accuracy 0.24673334
epoch 16 : loss 2.2289531 ; accuracy 0.25251666
epoch 17 : loss 2.2242815 ; accuracy 0.25823334
epoch 18 : loss 2.2196307 ; accuracy 0.26403335
epoch 19 : loss 2.215 ; accuracy 0.27005
epoch 20 : loss 2.2103894 ; accuracy 0.27506667
epoch 21 : loss 2.2057972 ; accu